In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

In [ ]:
DATA_PATH = os.path.join('dataset')
actions = np.array(['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
sequence_length = 100
num_features = 126
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_path)).astype(int):
        frame_paths = [
            os.path.join(action_path, str(sequence), "{}.npy".format(frame_num))
            for frame_num in range(sequence_length)
        ]
        window = [np.load(frame_path) for frame_path in frame_paths]
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [ ]:
log_dir = os.path.join('LSTM_Logs')
callback = TensorBoard(log_dir=log_dir)
ACCURACY_THRESHOLD = 0.95

class MyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, monitor_metric='accuracy', threshold=0.95):
        super(MyCallback, self).__init__()
        self.monitor_metric = monitor_metric
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs={}): 
        current_metric_value = logs.get(self.monitor_metric)
        if current_metric_value is not None and current_metric_value > self.threshold:
            print(f"\nReached {self.threshold * 100:.2f}% {self.monitor_metric}, stopping training!")
            self.model.stop_training = True

tb_callback = TensorBoard(log_dir=log_dir)
my_callback = MyCallback(monitor_metric='accuracy', threshold=ACCURACY_THRESHOLD)

In [ ]:
model = Sequential()

model.add(BatchNormalization())
model.add(LSTM(16, return_sequences=True, input_shape=(sequence_length, num_features), activation='tanh', kernel_regularizer=l2(0.001)))

model.add(BatchNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))

model.add(BatchNormalization())
model.add(LSTM(16, return_sequences=True))

model.add(BatchNormalization())
model.add(LSTM(16, return_sequences=True))

model.add(Flatten())

model.add(Dense(2048, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1536, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(1024, activation='relu'))

model.add(Dense(actions.shape[0], activation='softmax'))

In [1]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'Adam' is not defined

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, callbacks=[tb_callback, my_callback, early_stopping])

In [ ]:
model.summary()

In [ ]:
res = model.predict(x_test)

In [ ]:
model.save('LSTM.h5')

In [ ]:
model.load_weights('LSTM.h5')

In [ ]:
yhat = model.predict(x_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Train Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

In [ ]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Test Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)